In [19]:
import glob
import json
import shutil
import os
promising_categories = ['Question Answering',
'Question Decomposition',
'Question Generation',
'Question Rewriting',
'Question Understanding',
'Section Classification',
'Explanation',
'Text Categorization',
'Paraphrasing',
'Toxic Language Detection',]

check_mono_eng = lambda doc: doc["Input_language"][0] == doc["Output_language"][0] == doc["Instruction_language"][0] == "English"
file_list = glob.glob("tasks/*")
for path in sorted(file_list):
    with open(path) as f:
        j = json.load(f)
        for categ in promising_categories:
            categ_dir = "task-splits/" +categ.lower().replace(" ", "_") + "/"
            if not os.path.exists(categ_dir):
                os.mkdir(categ_dir)

            if j["Categories"][0] == categ and check_mono_eng(j):
                newpath = categ_dir + os.path.basename(path)
                shutil.copyfile(path, newpath)


In [746]:
categs = set()
for path in sorted(glob.glob("tasks/*")):
    j = json.load(open(path))
    if len(j["Categories"]) != 1:
        print(j["Categories"])
    # print(c)
    # categs.update(c)

In [739]:
categs = sorted(list(categs))


In [34]:
import json
import os
import shutil
initial_tasks = [
    'task075_squad1.1_answer_generation.json',
    'task176_break_decompose_questions.json',
    'task192_hotpotqa_sentence_generation.json',
    'task223_quartz_explanation_generation.json',
    'task593_sciq_explanation_generation.json']
# os.mkdir("initial_task_set")

prefix = "tasks/"

for task in initial_tasks:
    shutil.copy(prefix+task, "initial_task_set")



In [23]:
import json
with open("translated_tasks/fin-translated-task192_hotpotqa_sentence_generation.json") as f:
    d_fin = json.load(f)
with open("initial_task_set/task192_hotpotqa_sentence_generation.json") as f:
    d_en = json.load(f)

In [45]:
import numpy as np
def wrap_print(s, max_len=150):
    if len(s) > max_len:
        print(s[:max_len])
        wrap_print(s[max_len:], max_len)
    else:
        print(s)

np.random.seed(1)
indices = np.random.choice(range(len(d_fin["Instances"])), 50)

def print_range(a,b):
    for i in indices[a:b]:
        wrap_print(d_en["Instances"][i]["input"])
        print()
        wrap_print(d_fin["Instances"][i]["input"])
        print()
        print()

In [46]:
print_range(6,9)

Ross Case (born 1 November 1951) is an Australian former tennis player. His career-high singles ranking was world no. 14. Question: Who had the highes
t singles ranking Tomáš Šmíd or Ross Case? Answer: Tomáš Šmíd

Ross Case (s. 1. marraskuuta 1951) on australialainen entinen tennispelaaja. Hänen uransa korkein sijoitus kaksinpelissä oli maailmanlistan 14. sija. 
Kysymys: Tomáš Šmíd vai Ross Case? Vastaus: Tom Cáme, joka oli maailmanmestari: Tomáš Šmíd


Getty Oil is an American oil marketing company with its origins as part of the large integrated oil company founded by the father of J. Paul Getty. Qu
estion: What was the American industrialist that founded an American oil marketing company named by a 1957 "Fortune" magazine? Answer: the richest liv
ing American

Getty Oil on yhdysvaltalainen öljyn markkinointiyritys, joka on osa suurta integroitua öljy-yhtiötä, jonka perusti J. Paul Gettyn isä. Kysymys: Minkä 
amerikkalaisen teollisuusmiehen, joka perusti amerikkalaisen öljyn markkino

# Count "Question:"-substrings in inpus

In [30]:
import json
import glob

is_monolingual = lambda doc: doc["Input_language"][0] == doc["Output_language"][0] == doc["Instruction_language"][0] == "English" and len(doc["Input_language"])==1

def yield_docs():
    tasks = glob.glob("tasks/*json")
    for task in sorted(tasks):
        with open(task) as f:
            d = json.load(f)
            if is_monolingual(d):
                yield task, d
res = {}

for task, doc in yield_docs():
    q = [s for s in doc["Instances"] if "Question:" in s["input"]]
    a = [s for s in doc["Instances"] if "Answer:" in s["input"]]
    res.update({
        task: {
            "has_q": len(q), 
            "total_len": len(doc["Instances"]),
            "has_a": len(a), 
            "total_len": len(doc["Instances"]),
            }
        })

# Categorize tasks to questions = {all, some, none}

In [31]:
q_in_all = list()
has_some_qs = list()
has_no_qs = list()
other = list()
for k, v in res.items():
    if v["has_q"] == v["total_len"]:
        res[k].update({"questions": "all"})
    elif v["has_q"] > 0:
        res[k].update({"questions": "some"})
        # has_some_qs.append(k)
    elif v["has_q"] == 0:
        res[k].update({"questions": "none"})
    else:
        assert False
        
    if v["has_a"] == v["total_len"]:
        res[k].update({"answers": "all"})
    elif v["has_a"] > 0:
        res[k].update({"answers": "some"})
        # has_some_qs.append(k)
    elif v["has_a"] == 0:
        res[k].update({"answers": "none"})

In [29]:
only_ans = list()
for k, v in res.items():
    if v["questions"] 
    if v["questions"] == "none" and v["answers"] == "all":
        only_ans.append(k)

print(len(only_ans), "tasks have no questions but only answers")


9

In [32]:

for k, v in res.items():
    res[k].update({
        "q_perc": v["has_q"] / v["total_len"],   
        "a_perc": v["has_a"] / v["total_len"]
        })

In [53]:
for i, (k,v) in enumerate(res.items()):
    # if v["a_perc"]> 0.01 and v["a_perc"] < 1.0 and v["q_perc"]> 0.01 and v["q_perc"] < 1.0:
    if v["a_perc"]< 0.01 and v["q_perc"]< 0.01:
        res[k].update({"low_perc_qa": True})
    else:
        res[k].update({"low_perc_qa": False})

# Categorical or not?

In [54]:
for task, doc in yield_docs():
    categories = set()
    for instance in doc["Instances"]:
        # for o in instance["output"]:
            # print(o)
        categories.update(instance["output"])
    res[task].update({"levels":len(categories)})


# Add categorical-label

In [55]:
import matplotlib.pyplot as plt


a = [(k,v) for (k, v) in res.items() if v["levels"] >= 30 ]
for k,v in a:
    res[k].update({"type": "free-form"})

a = [(k,v) for (k, v) in res.items() if (v["levels"] < 30 and v["levels"] >= 10)]
for k,v in a:
    res[k].update({"type": "maybe-categorical"})

a = [(k,v) for (k, v) in res.items() if v["levels"] < 10]
for k,v in a:
    res[k].update({"type": "categorical"})


In [57]:
with open("classifications_of_tasks.json", "w") as out:
    json.dump(res, out, indent=2)

In [58]:
# tmp = list()
# for k, v in res.items():
#     if v.get("type") == "categorical" and v.get("low_perc_qa") == True:
#         tmp.append(k)
# len(tmp)

# import shutil

# for p in tmp:
#     shutil.copy2(p, "tasks-with-low-qa-in-input/categorical/")



for k, v in res.items():
    if v.get("type") == "" and v.get("low_perc_qa") == True:
        tmp.append(k)
len(tmp)

import shutil

for p in tmp:
    shutil.copy2(p, "tasks-with-low-qa-in-input/categorical/")